# Data Preprocessing

In this notebook, the data is prepared for training. It contains the following steps:

- Remove duplicate lines
- Remove personas with equal performance.

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_parquet('data/agent_compt.parquet')

## Check inputs with 16 personas

We assume that these are always duplicates, but further analysis has shown that this is not true.

In [3]:
counts = data['input'].map(data['input'].value_counts())
filtered_df = data[counts == 16]
filtered_df

,dataset,category,input,agent,response,ground_truth,pred,score
2824,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a psychiatrist or therapist specializi...,The correct answer is **(A) Acting out**. Here...,A,A,1
2825,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a trained medical professional special...,The most likely defense mechanism the patient ...,A,A,1
2826,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a psychologist specializing in adolesc...,The most likely defense mechanism the patient ...,A,A,1
2827,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a licensed psychologist or psychiatris...,The best answer here is **(A) Acting out**. He...,A,A,1
2828,mmlu_pro,health,A 14-year-old girl is brought to the physician...,"You are a skilled mental health professional, ...",The answer is **(A) Acting out**. Here's why:\...,A,A,1
...,...,...,...,...,...,...,...,...
168787,mmlu_pro,law,Who said that “Jurisprudence is the eye of law...,"You are a legal scholar, perhaps a professor s...","The answer is **(C) Savigny**.\n\nThis maxim, ...",H,C,0
168788,mmlu_pro,law,Who said that “Jurisprudence is the eye of law...,"You are Salmond, a renowned Scottish legal sch...",The answer is **(A) Pound**. \n\n**Roscoe Pou...,H,A,0
168789,mmlu_pro,law,Who said that “Jurisprudence is the eye of law...,You are a legal scholar specializing in the hi...,The correct answer is **(G) Blackstone**. \n\n...,H,G,0
168790,mmlu_pro,law,Who said that “Jurisprudence is the eye of law...,You are a legal scholar with a deep understand...,The answer is **(A) Pound**. \n\nHere's why:\n...,H,A,0


Even though some agents seem to be duplicates, they are just very similar. Some seem to be very similar, yet not the same. So inputs with 16 agents seem to be part of the dataset.

## Remove trailing line breaks

Some rows contain agent strings that only differ by a double line break. We remove the double line breaks so the duplicates get identified correctly.

In [4]:
data['agent'] = data['agent'].str.replace('\\n\\n', '\\n', regex=True)
data['agent'] = data['agent'].str.rstrip()

## Swap 'None' with empty string

In [5]:
data = data.replace({None: ''})

## Get rid of chat markers

In [6]:
data['input'] = data['input'].apply(lambda x: x.replace('<end_of_turn>\n<start_of_turn>model\n', '').strip())

## Remove any duplicates

In [7]:
data['agent'] = data['agent'].str.split('\n\n').str[0]
def clean_agent(text):
    # Truncate after double new line
    text = text.split('\n\n')[0]

    # Remove Markdown text styles
    text = re.sub(r'(\*\*|\*|__|_)(.*?)\1', r'\2', text)

    # Remove multiple whitespaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Apply the function to the 'agent' column
data['agent'] = data['agent'].apply(clean_agent)
data_unique = data.drop_duplicates(subset=['input', 'agent'], keep='first')

## Check outcome

In [8]:
data_unique['input'].value_counts().unique()

array([15, 14, 13, 12, 11, 10,  8,  7,  6,  5,  4,  3])

In [9]:
data_unique[data_unique['input'] == "A 14-year-old girl is brought to the physician after her mother learned that she began having sexual intercourse with various partners 1 month ago. She does not use condoms or other contraception. The mother is concerned about her behavior. The patient's parents separated 3 months ago. She had been an honor student and excelled in sports and leadership positions at school before the separation. Since the separation, however, she has become sullen, defiant, and rebellious. She has begun smoking cigarettes, disobeying her curfew, and being truant from school. This patient is most likely using which of the following defense mechanisms?\n(A) Acting out\n(B) Intellectualization\n(C) Projection\n(D) Regression\n(E) Displacement\n(F) Rationalization\n(G) Denial\n(H) Repression\n(I) Sublimation\n(J) Reaction formation"]

,dataset,category,input,agent,response,ground_truth,pred,score
2824,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a psychiatrist or therapist specializi...,The correct answer is **(A) Acting out**. Here...,A,A,1
2825,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a trained medical professional special...,The most likely defense mechanism the patient ...,A,A,1
2826,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a psychologist specializing in adolesc...,The most likely defense mechanism the patient ...,A,A,1
2827,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a licensed psychologist or psychiatris...,The best answer here is **(A) Acting out**. He...,A,A,1
2828,mmlu_pro,health,A 14-year-old girl is brought to the physician...,"You are a skilled mental health professional, ...",The answer is **(A) Acting out**. Here's why:\...,A,A,1
2829,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a qualified psychiatrist or psychologi...,The most likely defense mechanism at play here...,A,A,1
2830,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a licensed therapist specializing in a...,The answer is **(A) Acting out**. Here's why:\...,A,A,1
2831,mmlu_pro,health,A 14-year-old girl is brought to the physician...,"You are a licensed medical professional, speci...",The most likely defense mechanism at play here...,A,A,1
2832,mmlu_pro,health,A 14-year-old girl is brought to the physician...,You are a medical professional specializing in...,The most likely defense mechanism at play here...,A,A,1
2833,mmlu_pro,health,A 14-year-old girl is brought to the physician...,"You are a qualified medical professional, like...",The answer is **(A) Acting out**. Here's why:\...,A,A,1


There are still inputs with more than 8 different personas.

In [10]:
print(f'Deleted rows: {len(data.index) - len(data_unique.index)}')

Deleted rows: 3413


## Check for empty lines

In [11]:
data_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171699 entries, 0 to 175111
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   dataset       171699 non-null  object
 1   category      171699 non-null  object
 2   input         171699 non-null  object
 3   agent         171699 non-null  object
 4   response      171699 non-null  object
 5   ground_truth  171699 non-null  object
 6   pred          171699 non-null  object
 7   score         171699 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 11.8+ MB


Empty agent column indicates no persona was used. This is not an actual empty line. These will not be deleted. All other columns contain the expected (full) amount of entries.

## Clean unusable instances

For the training to work, we need some sort of performance hierarchy between the agents. Thus, all instances will be deleted where either all or no agents are able to complete the task. We can not rank these agents since they all perform equally good or bad.

In [12]:
# Group rows by input and keep only if unique values in score is larger than 1 (as else, all would be the same)
data_filtered = data_unique[data_unique.groupby('input')['score'].transform(lambda x: x.nunique() > 1)]

## Some information about the data

In [13]:
print(f'Deleted rows: {len(data.index) - len(data_filtered.index)}')

Deleted rows: 111101


In [14]:
print(f'Remaining rows: {len(data_filtered.index)}')

Remaining rows: 64011


In [15]:
print(f"Avg. agents per prompt: {len(data_filtered) / len(data_filtered['input'].unique())}")

Avg. agents per prompt: 7.925095951467129


In [16]:
print(f"Number of winning agents: {len(data_filtered[data_filtered['score'] == 1])}")

Number of winning agents: 32784


In [17]:
print(f"Number of losing agents: {len(data_filtered[data_filtered['score'] == 0])}")

Number of losing agents: 31227


## Setup data for fine-tuning

In [18]:
fine_tune_data = data_filtered[data_filtered['score'] == 1]
fine_tune_data = fine_tune_data[fine_tune_data['agent'] != '']

In [19]:
finetune_task_string = 'Given the following task, print a suitable agent string that will answer it as good as possible: '
fine_tune_data['prompt'] = finetune_task_string + fine_tune_data['input'].str.strip() + "\n"
fine_tune_data['completion'] = " " + fine_tune_data['agent'].str.strip()

In [20]:
fine_tune_data = fine_tune_data[['prompt', 'completion']]
fine_tune_data

,prompt,completion
2,"Given the following task, print a suitable age...","You are a classicist, perhaps with a focus on..."
3,"Given the following task, print a suitable age...","You are a classical scholar, specializing in ..."
4,"Given the following task, print a suitable age...","You are a Latin expert, familiar with the nua..."
6,"Given the following task, print a suitable age...",You are a klassically trained historian speci...
7,"Given the following task, print a suitable age...",You are a scholar specializing in ancient Rom...
...,...,...
175077,"Given the following task, print a suitable age...","You are a detective, skilled in examining det..."
175079,"Given the following task, print a suitable age...",This is a tricky prompt because you're lookin...
175080,"Given the following task, print a suitable age...","You are a legal scholar, perhaps a professor ..."
175084,"Given the following task, print a suitable age...","You are a legal scholar, possibly a professor..."


In [21]:
fine_tune_data.to_parquet('data/agent_finetune.parquet', index=False)

## Setup training dataset to allow DPO training

In this section, the actual dataset for training, testing and evaluating the model will be constructed. This will be done according to the documentation here: https://huggingface.co/docs/trl/main/en/dataset_formats#preference

There are two possible formats standard and conversational, as well as implicit and explicit form. Please refer to the documentation linked above to see examples.

In the following code, the dataset is built in the so-called standard format with explicit prompts (which is recommended).

Training samples are tuples of $(x, y_w, y_l)$, where $y_w$ is a winning agent prompt string, $y_l$ is a losing agent prompt string and $x$ is the model input.

In [21]:
train_data = []
analysis_data = []
count = 0
for input_val, group in data_filtered.groupby('input'):
    # Group by input and get winning and losing outputs
    winning_agents = group[group['score'] == 1]
    losing_agents = group[group['score'] == 0]

    for idx_winning, winning in winning_agents.iterrows():
        for idx_losing, losing in losing_agents.iterrows():
            # As a sanity check I want to check that the inputs are actually the same
            if not winning['input'] == losing['input']:
                raise Exception("Illegal combination of winning and losing input")

            # Task string was updated in favor of the new option
            model_task = 'Given the following task, print a suitable agent string that will answer it as good as possible: '

            # Actual dataset for training the model using conversational, explicit format
            train_data.append({
                'category': winning['category'],
                'orig_prompt': input_val,
                'prompt': [{'role': 'user', 'content': f'{model_task} {input_val}'}], # or losing['input'] or winning['input']
                'chosen': [{'role': 'assistant', 'content': winning['agent']}],
                'rejected': [{'role': 'assistant', 'content': losing['agent']}]
            })

            # The list entries of columns prompt and chosen are difficult to analyze
            analysis_data.append({
                'category': winning['category'],
                'orig_prompt': input_val,
                'prompt': f'{model_task} {input_val}', # or losing['input'] or winning['input']
                'chosen': winning['agent'],
                'rejected': losing['agent']
            })

In [22]:
print(f'Training (and test/eval) samples: {len(train_data)}')

Training (and test/eval) samples: 85142


In [23]:
train_df = pd.DataFrame(train_data)
train_df.to_parquet('data/agent_prompt_cleaned.parquet', index=False)

analysis_df = pd.DataFrame(analysis_data)
analysis_df.to_parquet('data_analysis/agent_prompt_cleaned.parquet', index=False)

In [24]:
train_df.head()

,category,orig_prompt,prompt,chosen,rejected
0,philosophy,""" _Ad crumenam_ "" is a specific kind of\n(A) S...","[{'role': 'user', 'content': 'Given the follow...","[{'role': 'assistant', 'content': ''}]","[{'role': 'assistant', 'content': 'You are a l..."
1,philosophy,""" _Ad crumenam_ "" is a specific kind of\n(A) S...","[{'role': 'user', 'content': 'Given the follow...","[{'role': 'assistant', 'content': ''}]","[{'role': 'assistant', 'content': 'You are a s..."
2,philosophy,""" _Ad crumenam_ "" is a specific kind of\n(A) S...","[{'role': 'user', 'content': 'Given the follow...","[{'role': 'assistant', 'content': 'You are a c...","[{'role': 'assistant', 'content': 'You are a l..."
3,philosophy,""" _Ad crumenam_ "" is a specific kind of\n(A) S...","[{'role': 'user', 'content': 'Given the follow...","[{'role': 'assistant', 'content': 'You are a c...","[{'role': 'assistant', 'content': 'You are a s..."
4,philosophy,""" _Ad crumenam_ "" is a specific kind of\n(A) S...","[{'role': 'user', 'content': 'Given the follow...","[{'role': 'assistant', 'content': 'You are a c...","[{'role': 'assistant', 'content': 'You are a l..."


## Prepare test set

Testing is performed on out-of-training categories with over 1000 inputs in the test set.

In [25]:
train_df['category'].value_counts()

category
physics                      7307
chemistry                    6050
math                         5919
truthfulqa                   4433
engineering                  4220
                             ... 
swahili_english_proverbs       64
checkmate_in_one               56
anachronisms                   50
hindu_knowledge                37
medical_questions_russian      26
Name: count, Length: 72, dtype: int64

In [26]:
test_categories = ['biology', 'philosophy', 'psychology',
                   'social_iqa', 'arithmetic', 'elementary_math_qa', 'history', 'economics']

test_df = train_df[train_df['category'].isin(test_categories)]
train_df = train_df[~train_df['category'].isin(test_categories)]

test_analysis_df = analysis_df[analysis_df['category'].isin(test_categories)]
train_analysis_df = analysis_df[~analysis_df['category'].isin(test_categories)]
len(test_df['category'])

12962

In [27]:
len(test_df.orig_prompt.unique())

1293

In [28]:
test_df = test_df.drop(columns=['category', 'orig_prompt'])
train_df = train_df.drop(columns=['category', 'orig_prompt'])

In [29]:
test_df.to_parquet('data/agent_test.parquet', index=False)
train_df.to_parquet('data/agent_train.parquet', index=False)

test_analysis_df.to_parquet('data_analysis/agent_test.parquet', index=False)
train_analysis_df.to_parquet('data_analysis/agent_train.parquet', index=False)